In [ ]:
import numpy as np
import basix
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [ ]:
from basix import CellType, ElementFamily, LagrangeVariant, LatticeType
import numpy as np
import matplotlib.pyplot as plt
import bbax

In [ ]:
factor = 3.0
m_cm = 1/100
water = (np.array([15.2090, 24.4140]) * m_cm,
         np.array([[[5.5069 , 0.0   ],
                  [6.6227, 11.4080]],
                  [[3.4757 /factor,  0.0    /factor],
                   [3.7043 /factor,  6.7420 /factor]]]) * m_cm)

iron = (np.array([30.3110, 26.8760]) * m_cm,
        np.array([[[18.250, 0.0   ],
                   [6.4496, 21.3570]],
                  [[13.0890  /factor ,  0.0  /factor],
                   [-0.010523/factor ,7.0373 /factor]]]) * m_cm
)

void = (np.array([0.0, 0.0]),
        np.array([[[0.0, 0.0],
                   [0.0, 0.0]],
                  [[0.0, 0.0],    
                   [0.0, 0.0]]]))

source = np.array([0.097702, 0.45451])
zero_source = np.array([0.0, 0.0])

regions = [
    (20, *water, zero_source),
    (10, *water, source ),
    (50, *water, zero_source),
    (20, *iron,  zero_source),
    (10, *void,  zero_source),
    (10, *water, zero_source),
    (10, *void,  zero_source)
]
elements_per_cm = 1




In [ ]:

lagrange    = basix.create_element(ElementFamily.P, CellType.interval, degree= 3, lagrange_variant= LagrangeVariant.gll_warped)
A, b, nodes = bbax.fem_1d.Create_Matrix_From_Regions(lagrange, regions, elements_per_cm, N_max=5, bc = "reflective", energy_group= 0)

In [ ]:
import numpy as np
from scipy.sparse.linalg import spsolve, cg

x = spsolve(A.tocsr(), b)
# Example usage:
# Suppose you want to plot the solution at 1000 points:
elem_dofs, _ = bbax.fem_1d.create_dof_matrix_vertex_interior(lagrange, nodes)

x_plot = np.linspace(nodes[0], nodes[-1], 1000)
u_plot = bbax.fem_1d.interpolate_solution(x_plot, nodes, elem_dofs, x, lagrange)
plt.plot(x_plot, u_plot * 1e4)
plt.xlabel("x")
plt.ylabel("u(x)")
plt.title("Interpolated FEM solution")
plt.yscale('log')

In [ ]:
plt.figure(figsize=(10, 10))
plt.spy(A, markersize=1)